In [19]:
# Import dependency
import sqlalchemy
from config import pgpassword, aws_endpoint, SQLALCHEMY_DATABASE_URI
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, Table
from datetime import datetime

In [20]:
# create engine 
engine = create_engine(SQLALCHEMY_DATABASE_URI, echo=False)

# reflect database
Base = automap_base()
Base.prepare(engine, reflect=True)

In [21]:
#  Create a meta data object to hold the reflected table schema
metadata = MetaData()

In [22]:
# Create a table object and use 'autoload' and 'autoload_with' to define the columns from the table. 
table = Table('SwShSeries', metadata, autoload=True, autoload_with=engine)

In [23]:
table

Table('SwShSeries', MetaData(bind=None), Column('id', TEXT(), table=<SwShSeries>), Column('name', TEXT(), table=<SwShSeries>), Column('number', TEXT(), table=<SwShSeries>), Column('legalities', JSON(astext_type=Text()), table=<SwShSeries>), Column('images', JSON(astext_type=Text()), table=<SwShSeries>), Column('prices', JSON(astext_type=Text()), table=<SwShSeries>), Column('set_id', TEXT(), table=<SwShSeries>), Column('set_name', TEXT(), table=<SwShSeries>), Column('date', TIMESTAMP(), table=<SwShSeries>), schema=None)

In [24]:
# Get the column names using the 'keys()' method on the column object. 
table.columns.keys()

['id',
 'name',
 'number',
 'legalities',
 'images',
 'prices',
 'set_id',
 'set_name',
 'date']

In [25]:
# Create a session
session = Session(engine)

# Get all the entries where the collector number is equal to passed value
results = session.query(table).filter(table.c.number == '001/073').order_by(table.c.date.desc())

In [26]:
for entry in results:
    print(entry)

('swsh35-1', 'Venusaur V', '001/073', {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, {'small': 'https://images.pokemontcg.io/swsh35/1.png', 'large': 'https://images.pokemontcg.io/swsh35/1_hires.png'}, {'holofoil': {'low': 1.2, 'mid': 2.36, 'high': 19.95, 'market': 2.12}}, 'swsh35', "Champion's Path", datetime.datetime(2021, 7, 2, 10, 44, 39, 159706))
('swsh35-1', 'Venusaur V', '001/073', {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, {'small': 'https://images.pokemontcg.io/swsh35/1.png', 'large': 'https://images.pokemontcg.io/swsh35/1_hires.png'}, {'holofoil': {'low': 1.25, 'mid': 2.44, 'high': 19.95, 'market': 2.04}}, 'swsh35', "Champion's Path", datetime.datetime(2021, 7, 1, 0, 31, 58, 370502))
('swsh35-1', 'Venusaur V', '001/073', {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, {'small': 'https://images.pokemontcg.io/swsh35/1.png', 'large': 'https://images.pokemontcg.io/swsh35/1_hires.png'}, {'holofoil': {'low': 1.25, 'mid': 2.

In [27]:
# create graph_data from query result
graph_data={}

graph_data['date']=[]
for style in results[0].prices:
    graph_data[style]={}
    for price_category in results[0].prices[style]:
        graph_data[style][price_category]=[]

    
for entry in results:
    graph_data['date'].append(datetime.date(entry.date))
    for style in entry.prices:
        for price_category in results[0].prices[style]:
            graph_data[style][price_category].append(entry.prices[style][price_category])

In [31]:
graph_data

{'date': [datetime.date(2021, 7, 2),
  datetime.date(2021, 7, 1),
  datetime.date(2021, 6, 30),
  datetime.date(2021, 6, 30)],
 'holofoil': {'low': [1.2, 1.25, 1.25, 1.25],
  'mid': [2.36, 2.44, 2.44, 2.44],
  'high': [19.95, 19.95, 19.95, 19.95],
  'market': [2.12, 2.04, 2.04, 2.04]}}

## SQL Method

In [28]:
entries = engine.execute('SELECT * FROM public."SwShSeries" WHERE number = \'001/073\'')


In [29]:
print (entries)


In [30]:
for entry in entries:
    print(entry)

('swsh35-1', 'Venusaur V', '001/073', {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, {'small': 'https://images.pokemontcg.io/swsh35/1.png', 'large': 'https://images.pokemontcg.io/swsh35/1_hires.png'}, {'holofoil': {'low': 1.25, 'mid': 2.44, 'high': 19.95, 'market': 2.04}}, 'swsh35', "Champion's Path", datetime.datetime(2021, 6, 30, 21, 51, 53, 455003))
('swsh35-1', 'Venusaur V', '001/073', {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, {'small': 'https://images.pokemontcg.io/swsh35/1.png', 'large': 'https://images.pokemontcg.io/swsh35/1_hires.png'}, {'holofoil': {'low': 1.25, 'mid': 2.44, 'high': 19.95, 'market': 2.04}}, 'swsh35', "Champion's Path", datetime.datetime(2021, 6, 30, 22, 42, 25, 939452))
('swsh35-1', 'Venusaur V', '001/073', {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, {'small': 'https://images.pokemontcg.io/swsh35/1.png', 'large': 'https://images.pokemontcg.io/swsh35/1_hires.png'}, {'holofoil': {'low': 1.25, 'mid'